<a href="https://colab.research.google.com/github/444112029012/4112029012_cal/blob/main/data_collection/ai_dataset/selenium_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab 進行matplotlib繪圖時顯示繁體中文
# 下載台北思源黑體並命名taipei_sans_tc_beta.ttf，移至指定路徑
!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download

import matplotlib

# 改style要在改font之前
# plt.style.use('seaborn')

matplotlib.font_manager.fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
matplotlib.rc('font', family='Taipei Sans TC Beta')
from google.colab import files
import pandas as pd

def load_data() -> pd.DataFrame:
  uploaded = files.upload()

  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

  # 將上傳的檔案讀取到 pandas DataFrame 中
  # 假設上傳的檔案是 CSV 格式。如果是其他格式，您可能需要調整讀取函數（例如：pd.read_excel）
  file_name = next(iter(uploaded))
  df = pd.read_csv(file_name)

  # 顯示 DataFrame 的前 5 行
  display(df.head())
  return df
# df = load_data()
def save_df(df, filename = 'phishing_dataset_for_trainning'):
    """
    將指定的 DataFrame 存成 CSV 檔案。

    參數:
    - df: 要儲存的 DataFrame
    - filename: 存檔的檔名（例如 'output.csv'）

    功能:
    - 使用 UTF-8 with BOM 編碼避免中文亂碼
    - 不包含索引欄位
    """
    try:
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"✅ 資料已成功儲存至 '{filename}'")
    except Exception as e:
        print(f"❌ 儲存失敗：{e}")

--2025-11-18 09:26:11--  https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving drive.google.com (drive.google.com)... 142.250.101.113, 142.250.101.100, 142.250.101.138, ...
Connecting to drive.google.com (drive.google.com)|142.250.101.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_ [following]
--2025-11-18 09:26:11--  https://drive.usercontent.google.com/download?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.141.132, 2607:f8b0:4023:c0b::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.141.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20659344 (20M) [application/octet-stream]
Saving to: ‘TaipeiSansTCBeta-Regular.ttf’

TaipeiSansTCBeta-Re 100%[===================>]  19.70M  66.2MB/s    in 0.3s    

2025-11-18 

In [ ]:
%%capture
!pip install google-generativeai
!pip install selenium
!pip install webdriver-manager
!apt-get update
# !apt install chromium-chromedriver
# 2. 手動安裝與最新 WebDriver 相符的 Chrome 瀏覽器
#    這能確保版本匹配，避免啟動錯誤
# !wget -q https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/114.0.5735.90/win64/chrome-win64.zip
# !unzip -q chrome-win64.zip -d /bin/

In [3]:
%%capture
!pip install google-colab-selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.3 MB/s eta 0:00:00


In [ ]:
%%capture
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, WebDriverException
import os


from google_colab_selenium import Chrome

# --- 設定 ---
FILE_NAME = "/content/phishing_dataset_expansion_forEmbeddingModule_Gemini.csv"  # 你的資料集檔案名稱
NEW_FILE_NAME = '/content/phishing_dataset_expansion_forEmbeddingModule_Gemini_text.csv'
NEW_COLUMN_NAME = 'visible_text'                    # 我們要創建的新欄位
SAVE_INTERVAL = 10                                  # 每處理 N 筆資料就儲存一次，防止中斷
RENDER_WAIT_TIME = 3                                # 載入頁面後，等待 JS 渲染的秒數

def setup_driver():
    """初始化 Selenium WebDriver"""
    print("正在初始化 WebDriver...")
    options = Options()
    options.add_argument('--headless')  # 在背景執行，不開啟瀏覽器視窗
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")
    # --- [!!! 新增此區塊以提高安全性 !!!] ---
    # 設定瀏覽器的下載行為
    prefs = {
        # "download.default_directory": "/dev/null",  # 將下載指向一個無效位置 (Linux/macOS)
        "download.default_directory": "NUL",      # (如果是 Windows, 用這個)
        "download.prompt_for_download": False,      # 不詢問下載位置
        "download.directory_upgrade": True,
        "profile.default_content_settings.popups": 0, # 封鎖彈出視窗
        "safebrowsing.enabled": False,                # 關閉安全瀏覽 (避免它干擾爬蟲)
        "profile.default_content_setting_values.automatic_downloads": 2 # *** 關鍵：禁止自動下載 ***
    }
    options.add_experimental_option("prefs", prefs)
    # --- [安全設定結束] ---
    try:
        driver = Chrome(options=options)
        # 設定頁面載入和腳本執行的超時時間
        driver.set_page_load_timeout(30)  # 30秒內頁面必須載入
        driver.set_script_timeout(10)   # 10秒內腳本必須執行完畢
        print("WebDriver 初始化完成。")
        return driver
    except Exception as e:
        print(f"WebDriver 初始化失敗: {e}")
        print("請確保你已安裝 Google Chrome 瀏覽器。")
        return None

def fetch_visible_text(driver, url):
    """
    (爬取文本的方法)
    使用 Selenium 爬取指定 URL 的可見文本 (document.body.innerText)
    """
    if not isinstance(url, str) or not url.strip():
        return "FETCH_ERROR: Invalid URL"

    # 確保 URL 有 http/https 協議頭
    if not url.startswith('http://') and not url.startswith('https://'):
        url = 'http://' + url

    try:
        driver.get(url)
        page_source_lower = driver.page_source.lower()

        if "dns_probe_finished_nxdomain" in page_source_lower or "err_name_not_resolved" in page_source_lower:
            print(f"  [Info] 網站不存在 (DNS): {url}")
            return "FETCH_ERROR: DNS_PROBE_FINISHED_NXDOMAIN"
        if "err_connection_refused" in page_source_lower:
            print(f"  [Info] 連線被拒: {url}")
            return "FETCH_ERROR: ERR_CONNECTION_REFUSED"
        if "err_connection_timed_out" in page_source_lower:
            print(f"  [Info] 連線超時: {url}")
            return "FETCH_ERROR: ERR_CONNECTION_TIMED_OUT"

        # 等待固定的秒數，讓 JavaScript 有時間渲染頁面
        time.sleep(RENDER_WAIT_TIME)

        # 執行 JS 來獲取 innerText
        text = driver.execute_script("return document.body.innerText;")

        if text is None:
             return "FETCH_EMPTY: 頁面未回傳可見文本"

        # --- [!!! 這是你要求的新清潔邏輯 !!!] ---

        # 1. 將文本按 "換行" 拆分為陣列
        lines = text.split('\n')

        # 2. 遍歷每一行，去除前後空白，並只保留 "非空" 的行
        non_empty_lines = [line.strip() for line in lines if line.strip()]

        # 3. 如果過濾後沒有任何內容，回傳 EMPTY
        if not non_empty_lines:
            if "<frame" in page_source_lower:
                return "FETCH_EMPTY: 頁面為 <frame> 結構"
            return "FETCH_EMPTY: 頁面未回傳可見文本 (清潔後)"

        # 4. 將乾淨的行用 "單一空格" 串接成一個字串
        cleaned_text = '\n'.join(non_empty_lines)
        print(f'清潔後的文本: {cleaned_text}')
        return cleaned_text
        # --- [清潔邏輯結束] ---


    except TimeoutException:
        print(f"  [Error] 頁面載入超時: {url}")
        return "FETCH_ERROR: Page load timed out"
    except WebDriverException as e:
        error_msg = str(e).split('\n')[0]
        print(f"  [Error] WebDriver 錯誤: {error_msg}")
        return f"FETCH_ERROR: {error_msg}"
    except Exception as e:
        error_msg = str(e).split('\n')[0]
        print(f"  [Error] 未知錯誤: {error_msg}")
        return f"FETCH_ERROR: Unknown error - {error_msg}"

def process_dataset(df, column_name, file_to_save):
    """
    (主要處理方法)
    遍歷 DataFrame，呼叫爬蟲，並即時更新資料集
    """
    total_rows = len(df)
    print(f"總共 {total_rows} 筆資料需要處理...")
    nb_driver = 200
    # 遍歷 DataFrame 的每一行
    driver = setup_driver()
    try:
        for index, row in df.iterrows():

            # 檢查 'visible_text' 欄位是否為空 (pd.isna) 或為空字串
            # 如果已有內容，則跳過，實現「斷點續爬」
            if pd.isna(row[column_name]) or row[column_name] == "":
                url = row['url']
                print(f"正在處理第 {index+1}/{total_rows} 筆: {url}")
                if (index+1) % nb_driver == 0:
                    driver.quit()
                    driver = setup_driver()
                # (呼叫爬取文本的方法)
                visible_text = fetch_visible_text(driver, url)

                # (將文本回傳後，直接更新資料集)
                # 使用 .at 來精確、快速地更新單一儲存格
                df.at[index, column_name] = visible_text

                # 每隔 N 筆資料就儲存一次檔案
                if (index + 1) % SAVE_INTERVAL == 0:
                    print(f"--- 已處理 {index+1} 筆，正在儲存進度... ---")
                    df.to_csv(file_to_save, index=False)

            else:
                # 如果該欄位已有資料，則跳過
                print(f"跳過第 {index+1}/{total_rows} 筆 (已有資料)")

        print("進度已儲存。")
        print("所有資料處理完畢。")
        return df, driver
    except KeyboardInterrupt:

        raise KeyboardInterrupt

# --- 主程式執行 ---
if __name__ == "__main__":

    # 1. 讀入資料集
    if os.path.exists(NEW_FILE_NAME):
        # 如果新檔案已存在，表示我們上次跑到一半，從這裡繼續
        print(f"找到進度檔: {NEW_FILE_NAME}。正在載入並繼續任務...")
        try:
            df = pd.read_csv(NEW_FILE_NAME)
        except Exception as e:
            print(f"讀取 {NEW_FILE_NAME} 時發生錯誤: {e}。")
            print(f"將嘗試從原始檔案 {FILE_NAME} 重新開始。")
            try:
                df = pd.read_csv(FILE_NAME)
            except Exception as e_orig:
                 print(f"連讀取 {FILE_NAME} 都失敗: {e_orig}。程式終止。")
                 exit()
    else:
        # 如果新檔案不存在，表示這是第一次執行，從原始檔案載入
        print(f"找不到進度檔。正在從原始檔案 {FILE_NAME} 載入...")
        try:
            df = pd.read_csv(FILE_NAME)
            print(f"成功讀取資料集: {FILE_NAME}")
        except FileNotFoundError:
            print(f"錯誤: 找不到原始檔案 '{FILE_NAME}'。請確認檔案名稱與路徑。")
            exit()
        except Exception as e:
            print(f"讀取 {FILE_NAME} 時發生錯誤: {e}")
            exit()

    # 2. 若資料集不存在我們需要的欄位，就先創建欄位
    if NEW_COLUMN_NAME not in df.columns:
        print(f"找不到欄位 '{NEW_COLUMN_NAME}'，正在新增...")
        df[NEW_COLUMN_NAME] = ""  # 初始化為空字串
    else:
        print(f"找到欄位 '{NEW_COLUMN_NAME}'，將繼續處理未填滿的資料。")
        # 將可能的 NaN (Not a Number) 轉為空字串，方便後續判斷
        df[NEW_COLUMN_NAME] = df[NEW_COLUMN_NAME].fillna("")

    try:
        # 3. 傳入資料集進行處理
        df_updated= process_dataset(df, NEW_COLUMN_NAME, NEW_FILE_NAME)

        # (最後回傳新資料集) - 並儲存最終版本
        print("正在儲存最終資料集...")
        df_updated.to_csv(NEW_FILE_NAME, index=False)
        print("任務完成！")
    except KeyboardInterrupt:
        # 如果使用者手動中斷 (Ctrl+C)
        print("\n偵測到手動中斷... 正在儲存目前進度...")
        df.to_csv(NEW_FILE_NAME, index=False)
    except Exception as e:
        print(f"主程式發生錯誤: {e}")
        print("正在嘗試儲存目前進度...")
        df.to_csv(NEW_FILE_NAME, index=False)
    finally:
        # 無論如何都要關閉瀏覽器
        print("正在關閉 WebDriver...")

找到進度檔: /content/phishing_dataset_expansion_forEmbeddingModule_Gemini_text.csv。正在載入並繼續任務...
找到欄位 'visible_text'，將繼續處理未填滿的資料。
總共 100000 筆資料需要處理...
正在初始化 WebDriver...


/tmp/ipython-input-610084781.py:174: DtypeWarning: Columns (2,3,4,5,6,7,8,9,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(NEW_FILE_NAME)


<IPython.core.display.Javascript object>

串流輸出內容已截斷至最後 5000 行。
旁遮普文 (古木基)
旁遮普文 (沙木基)
桑戈文
桑塔利文 (拉丁字母)
桑塔利文 (桑塔利文)
泰文
泰米爾文
泰盧固文
海地克里奧文
烏干達文
烏克蘭文
烏茲別克文
烏爾都文
烏德穆爾特文
班巴拉文
納瓦特爾文 (東瓦斯特克)
索馬里文
草原馬里文
貢根文
馬耳他文
馬來文
馬來文 (爪夷)
馬其頓文
馬拉加斯文
馬拉地文
馬拉雅拉姆文
馬紹爾文
馬都拉文
馬爾瓦第文
高棉文
宿霧文
捷克文
曼島文
望加錫文
梅泰文 (曼尼普爾文)
梵文
紹納文
荷蘭文
通布卡文
博杰普爾文
喬治亞文
奥克文
富拉尼文
斐濟文
斯瓦希里文
斯洛伐克文
斯洛維尼亞文
景頗文
欽邦哈卡文
猶加敦馬雅文
菲律賓文
越南文
隆地文
雅庫特文
塔吉克文
塔馬塞特文
塔馬塞特文 (提非納文)
塞爾維亞文
奧塞提亞文
奧羅莫文
意第緒文
愛沙尼亞文
愛爾蘭文
楚瓦什文
楚克文
溫達文
瑞典文
瑟索托文
粵語
義大利文
葡萄牙文 (巴西)
葡萄牙文 (葡萄牙)
蒂夫文
蒂格里亞文
道格里文
達利文
圖瓦文
圖陸文
廓博羅克文
瑪姆文
維吾爾文
蒙古文
豪沙文
寮文
德文
德頓文
撣文
模里西斯克里奧文
歐利亞文 (奧里雅文)
緬甸文
盧安達文
盧森堡文
盧奧文
錫蘭文
鮑勒文
優魯巴文
賽席爾克里奧爾文
邁蒂利文
韓文
薩米文 (北)
薩波特克文
薩摩亞文
藏文
豐文
羅姆文
羅馬尼亞文
蘇格蘭蓋爾文
蘇蘇文
韃靼文
由「翻譯」技術提供
正在處理第 25918/100000 筆: https://www.hybridband.com
清潔後的文本: H Y B R I D
HOME
CALLING YOUR NAME
NEWS
BLACK HALO
LISTEN
SHOP
VIDEO
MUSIC
BIO
CONTACT
By using this website, you agree to our use of cookies. We use cookies to provide you with a great experience and to help our website run effectively.
OK
CALLING YOUR NAME
THE NEW SINGLE
BUY / STREAM
CALLING YOUR NAME (

<IPython.core.display.Javascript object>

串流輸出內容已截斷至最後 5000 行。
YouTube
HELP US SPREAD THE WORD!
Please help us spread the news about our awesome products.
Tweet  Share  Pin it  Fancy
CURRENCY
USD $
Copyright © 2025 Feeding Tube Holder.
Powered by Shopify
★ REVIEWS
--- 已處理 26120 筆，正在儲存進度... ---
正在處理第 26121/100000 筆: https://www.mandaeanunion.org
清潔後的文本: عربي English
Home
News
Culture
History
Contact Us
Views
Human Rights
Donate
MAU Editorial
The Mandaeans are part of the indigenous people of Iraq. They are a religious and ethnic minority that is facing annihilation. All articles of the UN declaration that apply to the protection of the indigenous, ethnic and religious minorities are compatible with their situation and should be applied. The Sabian Mandaeans in Iraq are looking for protection .
Default
Title
Date
المزيد
Although Mandaeans neglected to document their history and have lost…
المزيد
The Mandaeans have for the last 2000 years resided along…
المزيد
Mandaean Moral Values "And when ye , my chosen ones give…
المزيد
275 A

<IPython.core.display.Javascript object>

WebDriver 初始化完成。
清潔後的文本: 404 Not Found
nginx/1.18.0 (Ubuntu)
--- 已處理 26200 筆，正在儲存進度... ---
正在處理第 26201/100000 筆: https://www.aelp.org.uk
清潔後的文本: Skip to content
Open Search Modal
JOIN US
Open Menu
Welcome to AELP
We are a national membership body, proudly representing the skills sector.
WHY JOIN US?
Why you should join us…
Get expert advice in a safe place
Your chance to ensure you know everything you need to know. Access to a friendly, knowledgeable team on call to answer your sector specific questions.
Be part of a supportive community
Link up with thousands of people working in the skills sector and enjoy networking opportunities at our conferences, summits and member-exclusive special interest groups and sector forums.
Support your sector
Your chance to make a difference: be part of advocating for policy changes to deliver a more sustainable skills sector. Get your voice heard through regular consultation events, surveys and our mini commissions.
Make cost savings
Our membership is

# **AI**

In [ ]:
%%capture
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, WebDriverException
import os
import numpy as np
from urllib3.exceptions import MaxRetryError, ReadTimeoutError

from google_colab_selenium import Chrome

# --- 設定 ---
FILE_NAME = "/content/phishing_dataset_expansion_2_Gemini.csv"  # 你的資料集檔案名稱
NEW_FILE_NAME = '/content/phishing_dataset_expansion_2_Gemini_text.csv'
NEW_COLUMN_NAME = 'visible_text'                    # 我們要創建的新欄位
SAVE_INTERVAL = 10                                  # 每處理 N 筆資料就儲存一次，防止中斷
RENDER_WAIT_TIME = 3                                # 載入頁面後，等待 JS 渲染的秒數

def setup_driver():
    """初始化 Selenium WebDriver"""
    print("正在初始化 WebDriver...")
    options = Options()
    options.add_argument('--headless')  # 在背景執行，不開啟瀏覽器視窗
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")
    options.page_load_strategy = 'eager'
    # --- [!!! 新增此區塊以提高安全性 !!!] ---
    # 設定瀏覽器的下載行為
    prefs = {
        # "download.default_directory": "/dev/null",  # 將下載指向一個無效位置 (Linux/macOS)
        "download.default_directory": "NUL",      # (如果是 Windows, 用這個)
        "download.prompt_for_download": False,      # 不詢問下載位置
        "download.directory_upgrade": True,
        "profile.default_content_settings.popups": 0, # 封鎖彈出視窗
        "safebrowsing.enabled": False,                # 關閉安全瀏覽 (避免它干擾爬蟲)
        "profile.default_content_setting_values.automatic_downloads": 2 # *** 關鍵：禁止自動下載 ***
    }
    options.add_experimental_option("prefs", prefs)
    # --- [安全設定結束] ---
    try:
        driver = Chrome(options=options)
        # 設定頁面載入和腳本執行的超時時間
        driver.set_page_load_timeout(30)  # 30秒內頁面必須載入
        driver.set_script_timeout(10)   # 10秒內腳本必須執行完畢
        print("WebDriver 初始化完成。")
        return driver
    except Exception as e:
        print(f"WebDriver 初始化失敗: {e}")
        print("請確保你已安裝 Google Chrome 瀏覽器。")
        return None

def fetch_visible_text(driver, url):
    """
    (爬取文本的方法)
    使用 Selenium 爬取指定 URL 的可見文本 (document.body.innerText)
    """
    if not isinstance(url, str) or not url.strip():
        return "FETCH_ERROR: Invalid URL"

    # 確保 URL 有 http/https 協議頭
    if not url.startswith('http://') and not url.startswith('https://'):
        url = 'http://' + url

    try:
        driver.get(url)
        page_source_lower = driver.page_source.lower()

        if "dns_probe_finished_nxdomain" in page_source_lower or "err_name_not_resolved" in page_source_lower:
            print(f"  [Info] 網站不存在 (DNS): {url}")
            return "FETCH_ERROR: DNS_PROBE_FINISHED_NXDOMAIN"
        if "err_connection_refused" in page_source_lower:
            print(f"  [Info] 連線被拒: {url}")
            return "FETCH_ERROR: ERR_CONNECTION_REFUSED"
        if "err_connection_timed_out" in page_source_lower:
            print(f"  [Info] 連線超時: {url}")
            return "FETCH_ERROR: ERR_CONNECTION_TIMED_OUT"

        # 等待固定的秒數，讓 JavaScript 有時間渲染頁面
        time.sleep(RENDER_WAIT_TIME)

        # 執行 JS 來獲取 innerText
        text = driver.execute_script("return document.body.innerText;")

        if text is None:
             return "FETCH_EMPTY: 頁面未回傳可見文本"

        # --- [!!! 這是你要求的新清潔邏輯 !!!] ---

        # 1. 將文本按 "換行" 拆分為陣列
        lines = text.split('\n')

        # 2. 遍歷每一行，去除前後空白，並只保留 "非空" 的行
        non_empty_lines = [line.strip() for line in lines if line.strip()]

        # 3. 如果過濾後沒有任何內容，回傳 EMPTY
        if not non_empty_lines:
            if "<frame" in page_source_lower:
                return "FETCH_EMPTY: 頁面為 <frame> 結構"
            return "FETCH_EMPTY: 頁面未回傳可見文本 (清潔後)"

        # 4. 將乾淨的行用 "單一空格" 串接成一個字串
        cleaned_text = '\n'.join(non_empty_lines)
        print(f'清潔後的文本: {cleaned_text}')
        return cleaned_text
        # --- [清潔邏輯結束] ---


    except TimeoutException:
        print(f"  [Error] 頁面載入超時: {url}")
        return "FETCH_ERROR: Page load timed out"
    except WebDriverException as e:
        error_msg = str(e).split('\n')[0]
        print(f"  [Error] WebDriver 錯誤: {error_msg}")
        return f"FETCH_ERROR: {error_msg}"

    except Exception as e:
        error_msg = str(e).split('\n')[0]
        print(f"  [Error] 未知錯誤: {error_msg}")
        return f"FETCH_ERROR: Unknown error - {error_msg}"

def process_dataset(df, column_name, file_to_save):
    """
    (主要處理方法)
    遍歷 DataFrame，呼叫爬蟲，並即時更新資料集
    """
    total_rows = len(df)
    print(f"總共 {total_rows} 筆資料需要處理...")
    nb_driver = 200
    # 遍歷 DataFrame 的每一行
    driver = setup_driver()
    try:
        for index, row in df.iterrows():

            # 檢查 'visible_text' 欄位是否為空 (pd.isna) 或為空字串
            # 如果已有內容，則跳過，實現「斷點續爬」
            if pd.isna(row[column_name]) or row[column_name] == "":
                url = row['url']
                print(f"正在處理第 {index+1}/{total_rows} 筆: {url}")
                if (index+1) % nb_driver == 0:
                    driver.quit()
                    driver = setup_driver()
                # (呼叫爬取文本的方法)
                visible_text = fetch_visible_text(driver, url)
                if isinstance(visible_text, str) and visible_text.startswith("FETCH_ERROR"):
                    if "Read timed out" in visible_text or "HTTPConnectionPool" in visible_text:
                        print("🚨 偵測到 Driver 連線逾時，正在重啟 Driver...")
                        if driver:
                            driver.quit()
                        driver = setup_driver()
                        print('重試一次...')
                        visible_text = fetch_visible_text(driver, url)
                # (將文本回傳後，直接更新資料集)
                # 使用 .at 來精確、快速地更新單一儲存格
                df.at[index, column_name] = visible_text

                # 每隔 N 筆資料就儲存一次檔案
                if (index + 1) % SAVE_INTERVAL == 0:
                    print(f"--- 已處理 {index+1} 筆，正在儲存進度... ---")
                    df.to_csv(file_to_save, index=False)

            else:
                # 如果該欄位已有資料，則跳過
                print(f"跳過第 {index+1}/{total_rows} 筆 (已有資料)")

        print("進度已儲存。")
        print("所有資料處理完畢。")
        return df, driver
    except KeyboardInterrupt:

        raise KeyboardInterrupt

# --- 主程式執行 ---
if __name__ == "__main__":

    # 1. 讀入資料集
    if os.path.exists(NEW_FILE_NAME):
        # 如果新檔案已存在，表示我們上次跑到一半，從這裡繼續
        print(f"找到進度檔: {NEW_FILE_NAME}。正在載入並繼續任務...")
        try:
            df = pd.read_csv(NEW_FILE_NAME)
        except Exception as e:
            print(f"讀取 {NEW_FILE_NAME} 時發生錯誤: {e}。")
            print(f"將嘗試從原始檔案 {FILE_NAME} 重新開始。")
            try:
                df = pd.read_csv(FILE_NAME)
            except Exception as e_orig:
                 print(f"連讀取 {FILE_NAME} 都失敗: {e_orig}。程式終止。")
                 exit()
    else:
        # 如果新檔案不存在，表示這是第一次執行，從原始檔案載入
        print(f"找不到進度檔。正在從原始檔案 {FILE_NAME} 載入...")
        try:
            df = pd.read_csv(FILE_NAME)
            print(f"成功讀取資料集: {FILE_NAME}")
        except FileNotFoundError:
            print(f"錯誤: 找不到原始檔案 '{FILE_NAME}'。請確認檔案名稱與路徑。")
            exit()
        except Exception as e:
            print(f"讀取 {FILE_NAME} 時發生錯誤: {e}")
            exit()

    # 2. 若資料集不存在我們需要的欄位，就先創建欄位
    if 'gemini_status' not in df.columns:
        new_columns = [
                    'gemini_status', #新增的狀態欄位
                    'fetch_status', # 新增的狀態欄位
                    'creates_urgency', 'uses_threats', 'requests_sensitive_info',
                    'offers_unrealistic_rewards', 'has_spelling_grammar_errors',
                    'impersonated_brand', 'language_professionalism_score',
                    'overall_phishing_likelihood_score', 'summary_of_intent'
                ]
        for col in new_columns:
            df[col] = np.nan
    if NEW_COLUMN_NAME not in df.columns:
        print(f"找不到欄位 '{NEW_COLUMN_NAME}'，正在新增...")
        df[NEW_COLUMN_NAME] = ""  # 初始化為空字串
    else:
        print(f"找到欄位 '{NEW_COLUMN_NAME}'，將繼續處理未填滿的資料。")
        # 將可能的 NaN (Not a Number) 轉為空字串，方便後續判斷
        df[NEW_COLUMN_NAME] = df[NEW_COLUMN_NAME].fillna("")

    try:
        # 3. 傳入資料集進行處理
        df_updated= process_dataset(df, NEW_COLUMN_NAME, NEW_FILE_NAME)

        # (最後回傳新資料集) - 並儲存最終版本
        print("正在儲存最終資料集...")
        df_updated.to_csv(NEW_FILE_NAME, index=False)
        print("任務完成！")
    except KeyboardInterrupt:
        # 如果使用者手動中斷 (Ctrl+C)
        print("\n偵測到手動中斷... 正在儲存目前進度...")
        df.to_csv(NEW_FILE_NAME, index=False)
    except Exception as e:
        print(f"主程式發生錯誤: {e}")
        print("正在嘗試儲存目前進度...")
        df.to_csv(NEW_FILE_NAME, index=False)
    finally:
        # 無論如何都要關閉瀏覽器
        print("正在關閉 WebDriver...")

# [Error] 未知錯誤: HTTPConnectionPool(host='localhost', port=51371): Read timed out. (read timeout=120)

找到進度檔: /content/phishing_dataset_expansion_2_Gemini_text.csv。正在載入並繼續任務...
找到欄位 'visible_text'，將繼續處理未填滿的資料。
總共 85795 筆資料需要處理...
正在初始化 WebDriver...


/tmp/ipython-input-1814848566.py:184: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(NEW_FILE_NAME)


<IPython.core.display.Javascript object>

WebDriver 初始化完成。
跳過第 1/85795 筆 (已有資料)
跳過第 2/85795 筆 (已有資料)
跳過第 3/85795 筆 (已有資料)
跳過第 4/85795 筆 (已有資料)
跳過第 5/85795 筆 (已有資料)
跳過第 6/85795 筆 (已有資料)
跳過第 7/85795 筆 (已有資料)
跳過第 8/85795 筆 (已有資料)
跳過第 9/85795 筆 (已有資料)
跳過第 10/85795 筆 (已有資料)
跳過第 11/85795 筆 (已有資料)
跳過第 12/85795 筆 (已有資料)
跳過第 13/85795 筆 (已有資料)
跳過第 14/85795 筆 (已有資料)
跳過第 15/85795 筆 (已有資料)
跳過第 16/85795 筆 (已有資料)
跳過第 17/85795 筆 (已有資料)
跳過第 18/85795 筆 (已有資料)
跳過第 19/85795 筆 (已有資料)
跳過第 20/85795 筆 (已有資料)
跳過第 21/85795 筆 (已有資料)
跳過第 22/85795 筆 (已有資料)
跳過第 23/85795 筆 (已有資料)
跳過第 24/85795 筆 (已有資料)
跳過第 25/85795 筆 (已有資料)
跳過第 26/85795 筆 (已有資料)
跳過第 27/85795 筆 (已有資料)
跳過第 28/85795 筆 (已有資料)
跳過第 29/85795 筆 (已有資料)
跳過第 30/85795 筆 (已有資料)
跳過第 31/85795 筆 (已有資料)
跳過第 32/85795 筆 (已有資料)
跳過第 33/85795 筆 (已有資料)
跳過第 34/85795 筆 (已有資料)
跳過第 35/85795 筆 (已有資料)
跳過第 36/85795 筆 (已有資料)
跳過第 37/85795 筆 (已有資料)
跳過第 38/85795 筆 (已有資料)
跳過第 39/85795 筆 (已有資料)
跳過第 40/85795 筆 (已有資料)
跳過第 41/85795 筆 (已有資料)
跳過第 42/85795 筆 (已有資料)
跳過第 43/85795 筆 (已有資料)
跳過第 44/85795 筆 (已有資料)
跳過第 45/85795 筆 (已有資料)
跳過

正在初始化 WebDriver...


...

<IPython.core.display.Javascript object>

WebDriver 初始化完成。
重試一次...
清潔後的文本: Your book | Our mission
There's a book inside everyone. So what's your story?
About Us
About Aly
Aly's Books
My Mum Says "Blah Blah Blah"
Beating the Bullies
When I Left
I Love You Now Until Forever
Never too big for a cuddle
Mummy Monster
Aly's Media
Newsletters
Aly's Books TV
Testimonials
Aly's News
Aly's Events
Aly's Photos
Aly Supports
Aly's Blog
Letters To Aly
Authors
Authors
Author's Media
Upcoming Authors
Illustrators
Our Services
Testimonials
Events
Contact Us
About Us
Why Aly’s Books is Right for You
As an independent book publishing company, Aly’s Books offers a highly professional and individualised approach. The team at Aly’s Books pride themselves on their dedication to helping any writer publish his/her work. Being located in Melbourne, Victoria’s cultural centre, allows for more accessibility.
Aly’s Books offers a quick and affordable publishing system that is designed to nurture and preserve the integrity of your work. Founded by Aly Wal

正在初始化 WebDriver...


<IPython.core.display.Javascript object>

WebDriver 初始化完成。
重試一次...
清潔後的文本: TinyURL
Visit TinyURL.com
URL Terminated
The TinyURL link you are visiting was used by its creator in violation of our Terms of Use. TinyURL has a strict no-abuse policy, so we apologize for the intrusion this user has caused you.
Violation of our Terms of Use include:
Copyright Infringement
Use or production of copyright-protected material without authorization.
Phishing
A fraudulent practice of sending links purporting to be from reputable companies in order to induce individuals to reveal personal and confidential information.
Malware
Software that is specifically designed to disrupt, damage, or gain unauthorized access to a computer system.
Spam
Any unwanted, unsolicited digital communication that is sent out in bulk.
Fraud or Money Making Scams
Wrongful or criminal deception intended to result in monetary gain.
...or any other use that is illegal, or otherwise violates our terms.
Note:
If you received this link from a spam message or email, please 